In [1]:
%pip install -U "leafmap[maplibre]" fiona shapely geopandas pydeck

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install dash openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import geopandas as gpd
import numpy as np
import pandas as pd
import leafmap.maplibregl as leafmap
import pydeck as pdk
import plotly.express as px
import plotly.graph_objects as go
from shapely import wkt
from dash import Dash, dcc, html, Input, Output
from ipywidgets import widgets
import folium
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from dash.exceptions import PreventUpdate
import openpyxl

In [4]:
Buildings_Period_1 = gpd.read_file(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period1.gpkg")
Buildings_Period_2 = gpd.read_file(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period2.gpkg")
Buildings_Period_3 = gpd.read_file(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period3.gpkg")
Buildings_Period_4 = gpd.read_file(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period4.gpkg")

In [5]:
def process_LST(gpkg_path):
    gdf = gpd.read_file(gpkg_path, encoding="utf-8")
    gdf = gdf.to_crs(epsg=4326)
    gdf['centroid'] = gdf.geometry.centroid
    gdf['lon'] = gdf['centroid'].x
    gdf['lat'] = gdf['centroid'].y
    return gdf.drop(columns=['centroid'])

lst_p1 = process_LST(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period1.gpkg")
lst_p2 = process_LST(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period2.gpkg")
lst_p3 = process_LST(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period3.gpkg")
lst_p4 = process_LST(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Maps\Suhu Bangunan_Period4.gpkg")


In [6]:
p1 = lst_p1
p2 = lst_p2
p3 = lst_p3
p4 = lst_p4

join_1 = gpd.sjoin(
    p1,
    p2[['geometry', 'Suhu_bangunan_period2mean', 'Suhu_bangunan_period2min', 'Suhu_bangunan_period2max']],
    how='left',
    predicate='intersects'
).drop(columns='index_right')

join_2 = gpd.sjoin(
    join_1,
    p3[['geometry', 'Suhu_bangunan_period3mean', 'Suhu_bangunan_period3min', 'Suhu_bangunan_period3max']],
    how='left',
    predicate='intersects'
).drop(columns='index_right')

final_join = gpd.sjoin(
    join_2,
    p4[['geometry', 'Suhu_bangunan_period4mean', 'Suhu_bangunan_period4min', 'Suhu_bangunan_period4max']],
    how='left',
    predicate='intersects'
).drop(columns='index_right')

final_join.head()

,full_id,Suhu_bangunan_period1mean,Suhu_bangunan_period1min,Suhu_bangunan_period1max,geometry,lon,lat,Suhu_bangunan_period2mean,Suhu_bangunan_period2min,Suhu_bangunan_period2max,Suhu_bangunan_period3mean,Suhu_bangunan_period3min,Suhu_bangunan_period3max,Suhu_bangunan_period4mean,Suhu_bangunan_period4min,Suhu_bangunan_period4max
0,r6645425,46.248209,45.879405,46.528829,"MULTIPOLYGON (((112.76777 -7.28279, 112.76784 ...",112.767820,-7.282958,47.424246,47.253449,47.455112,43.297019,42.977506,43.456029,53.657801,53.525516,53.675909
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.826388,41.623970,41.829051,50.587873,50.587727,50.628744
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.826388,41.623970,41.829051,50.632259,50.527912,50.784264
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.788519,41.565864,42.081985,50.587873,50.587727,50.628744
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.788519,41.565864,42.081985,50.632259,50.527912,50.784264


In [7]:
mean_cols = [
    'Suhu_bangunan_period1mean',
    'Suhu_bangunan_period2mean',
    'Suhu_bangunan_period3mean',
    'Suhu_bangunan_period4mean'
]
final_join['Total_mean_temperature'] = final_join[mean_cols].mean(axis=1)

min_cols = [
    'Suhu_bangunan_period1min',
    'Suhu_bangunan_period2min',
    'Suhu_bangunan_period3min',
    'Suhu_bangunan_period4min'
]
final_join['Total_min_temperature'] = final_join[min_cols].mean(axis=1)

max_cols = [
    'Suhu_bangunan_period1max',
    'Suhu_bangunan_period2max',
    'Suhu_bangunan_period3max',
    'Suhu_bangunan_period4max'
]
final_join['Total_max_temperature'] = final_join[max_cols].mean(axis=1)
final_join.head()


min_temp = final_join['Total_mean_temperature'].min()
max_temp = final_join['Total_mean_temperature'].max()

if max_temp - min_temp == 0:
    final_join['Building_Height'] = 50

else:
    final_join['Building_Height'] = ((final_join['Total_mean_temperature'] - min_temp) / (max_temp - min_temp)) * 100

print(len(final_join))
final_join.head()

15038


,full_id,Suhu_bangunan_period1mean,Suhu_bangunan_period1min,Suhu_bangunan_period1max,geometry,lon,lat,Suhu_bangunan_period2mean,Suhu_bangunan_period2min,Suhu_bangunan_period2max,Suhu_bangunan_period3mean,Suhu_bangunan_period3min,Suhu_bangunan_period3max,Suhu_bangunan_period4mean,Suhu_bangunan_period4min,Suhu_bangunan_period4max,Total_mean_temperature,Total_min_temperature,Total_max_temperature,Building_Height
0,r6645425,46.248209,45.879405,46.528829,"MULTIPOLYGON (((112.76777 -7.28279, 112.76784 ...",112.767820,-7.282958,47.424246,47.253449,47.455112,43.297019,42.977506,43.456029,53.657801,53.525516,53.675909,47.656819,47.408969,47.778970,96.833837
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.826388,41.623970,41.829051,50.587873,50.587727,50.628744,45.256496,45.045835,45.339785,52.574726
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.826388,41.623970,41.829051,50.632259,50.527912,50.784264,45.267592,45.030881,45.378665,52.779333
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.788519,41.565864,42.081985,50.587873,50.587727,50.628744,45.247028,45.031309,45.403018,52.400160
1,r6887321,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.788519,41.565864,42.081985,50.632259,50.527912,50.784264,45.258125,45.016355,45.441898,52.604767


In [ ]:
cat_cols = final_join.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = final_join.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Kolom kategorikal yang akan di-encode:", cat_cols)
print("Kolom numerik (tidak di-encode):", num_cols)

Kolom kategorikal yang akan di-encode: ['full_id']
Kolom numerik (tidak di-encode): ['Suhu_bangunan_period1mean', 'Suhu_bangunan_period1min', 'Suhu_bangunan_period1max', 'lon', 'lat', 'Suhu_bangunan_period2mean', 'Suhu_bangunan_period2min', 'Suhu_bangunan_period2max', 'Suhu_bangunan_period3mean', 'Suhu_bangunan_period3min', 'Suhu_bangunan_period3max', 'Suhu_bangunan_period4mean', 'Suhu_bangunan_period4min', 'Suhu_bangunan_period4max', 'Total_mean_temperature', 'Total_min_temperature', 'Total_max_temperature', 'Building_Height']


In [ ]:
if 'full_id' in final_join.columns:
	final_join.drop(columns=['full_id'], inplace=True)
final_join.head()

,Suhu_bangunan_period1mean,Suhu_bangunan_period1min,Suhu_bangunan_period1max,geometry,lon,lat,Suhu_bangunan_period2mean,Suhu_bangunan_period2min,Suhu_bangunan_period2max,Suhu_bangunan_period3mean,Suhu_bangunan_period3min,Suhu_bangunan_period3max,Suhu_bangunan_period4mean,Suhu_bangunan_period4min,Suhu_bangunan_period4max,Total_mean_temperature,Total_min_temperature,Total_max_temperature,Building_Height
0,46.248209,45.879405,46.528829,"MULTIPOLYGON (((112.76777 -7.28279, 112.76784 ...",112.767820,-7.282958,47.424246,47.253449,47.455112,43.297019,42.977506,43.456029,53.657801,53.525516,53.675909,47.656819,47.408969,47.778970,96.833837
1,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.826388,41.623970,41.829051,50.587873,50.587727,50.628744,45.256496,45.045835,45.339785,52.574726
1,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.826388,41.623970,41.829051,50.632259,50.527912,50.784264,45.267592,45.030881,45.378665,52.779333
1,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.788519,41.565864,42.081985,50.587873,50.587727,50.628744,45.247028,45.031309,45.403018,52.400160
1,43.233773,43.025358,43.514135,"MULTIPOLYGON (((112.77135 -7.2823, 112.7714 -7...",112.771164,-7.282413,45.377948,44.946285,45.387210,41.788519,41.565864,42.081985,50.632259,50.527912,50.784264,45.258125,45.016355,45.441898,52.604767


In [27]:
period_options ={
    "Periode 1" : "Suhu_bangunan_period1mean",
    "Periode 2" : "Suhu_bangunan_period2mean",
    "Periode 3" : "Suhu_bangunan_period3mean",
    "Periode 4" : "Suhu_bangunan_period4mean"
}

dropdown = widgets.Dropdown(
    options = period_options,
    value = "Suhu_bangunan_period1mean",
    description = "Periode",
    style = {"description_width" : "initial"},
)


m = leafmap.Map(
    center=[112.76360, -7.28230], zoom=11, style="liberty", pitch=45, bearing=0
)

data = final_join
paint_line = {
    "line-color": "white",
    "line-width": 2,
}

all_temp_colums = [col for col in data.columns if 'Suhu_bangunan_period' in col and 'mean' in col]
global_min_temp = data[all_temp_colums].min().min()
global_max_temp = data[all_temp_colums].max().max()
print(f"Global Minimum Temperature : {global_min_temp:.2f}")
print(f"Global Maximum Temperature : {global_max_temp:.2f}")

def get_global_color(min_global, max_global):
    stops = []
    colors = ["#ffffb2", "#fecc5c", "#fd8d3c", "#f03b20", "#bd0026"]
    interval = (max_global - min_global) / (len(colors) - 1)

    for i in range(len(colors)):
        stop_val = min_global + (i*interval)
        stops.append([stop_val, colors[i]])
    return stops

initial_global_stops = get_global_color(global_min_temp, global_max_temp)

paint_lst = {
    "fill-extrusion-color" : {
        "property" : dropdown.value,
        "stops" : initial_global_stops,
    },
    "fill-extrusion-height" : ["get", "Building_Height"],
    "fill-extrusion-opacity" : 0.9
}

m.add_gdf(data, layer_type="line", paint=paint_line, name="blocks-line")
m.add_gdf(data, layer_type="fill-extrusion", paint=paint_lst, name="LST")

# ADD LEGEND
legend_output = widgets.Output()

def create_legend(column_name):
    with legend_output:
        legend_output.clear_output()
        min_temp = data[column_name].min()
        max_temp = data[column_name].max()
        stops = get_global_color(min_temp, max_temp)

        display(widgets.HTML("<b>Building Temperature Legend (°C)</b>"))
        for i in range(len(stops) - 1):
            color = stops[i][1]
            val_min = stops[i][0]
            val_max = stops[i + 1][0]
            display(widgets.HTML(
                f"<div style='display: flex; align-items: center;'>"
                f"<div style='width: 20px; height: 20px; background-color: {color}; margin-right: 10px;'></div>"
                f"{val_min:.2f} – {val_max:.2f} °C"
                f"</div>"
            ))

def update_map(change):
    selected_column = change.new
    update_pain_lst = {
            "property" : selected_column,
            "stops" : initial_global_stops,
    }
    m.set_paint_property("LST", "fill-extrusion-color", update_pain_lst)
    create_legend(selected_column)

dropdown.observe(update_map, names="value")
display(dropdown, m, legend_output)
m.add_layer_control()

Global Minimum Temperature : 33.76
Global Maximum Temperature : 54.16


Dropdown(description='Periode', options={'Periode 1': 'Suhu_bangunan_period1mean', 'Periode 2': 'Suhu_bangunan…

Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

Output()

In [9]:
app = Dash(__name__)

# LOAD DATA
df = pd.read_excel(r"C:\Users\VINCENT\Downloads\Urban Heat 3D\Data\mansab_full_data.xlsx")

# CONVERT GEOMETRY COLUMN TO GEOMETRY SHAPE
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# CALCULATE CENTROID FOR EACH BUILDING
gdf['centroid'] = gdf['geometry'].centroid
gdf['centroid_lon'] = gdf['centroid'].x
gdf['centroid_lat'] = gdf['centroid'].y

# DEFINE FIXED COLOR SCALE
custom_color_scale = [
    [0.0, '#ffffb2'],
    [0.25, '#fecc5c'],
    [0.5, '#fd8d3c'],
    [0.75, '#f03b20'],
    [1.0, '#bd0026']
]


# CREATE FIRST PLOTLY MAP
fig_map = px.scatter_mapbox(
    gdf,
    lat="centroid_lat",
    lon="centroid_lon",
    color="Suhu_bangunan_period1mean",
    color_continuous_scale=custom_color_scale,
    range_color=(
        gdf['Suhu_bangunan_period1mean'].min(),
        gdf['Suhu_bangunan_period1mean'].max()
        ),
    hover_name="full_id",
    hover_data={
        "Suhu_bangunan_period1mean":":.2f",
        "Building_Height":":.2f",
        "centroid_lat":False,
        "centroid_lon":False
    },
    zoom=15
)

fig_map.update_layout(
    mapbox_style="carto-positron",
    margin={"r":0,"t":0,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Temp (°C)",
        thickness=20,
        len=0.75
    )
)

# DASHBOARD LAYOUT
app.layout = html.Div([
    html.H1("Manyar Sabrangan Subdistrict Building Temperature Dashboard", style={'textAlign': 'center', 'margin' : '20px', 'color': '#2c3e50'}),

    html.Div([
            dcc.Dropdown(
                id='period-dropdown',
                options=[
                    {'label': 'Periode 1 (Jan-Mar)', 'value': 'period1'},
                    {'label': 'Periode 2 (Apr-Jun)', 'value': 'period2'},
                    {'label': 'Periode 3 (Jul-Sep)', 'value': 'period3'},
                    {'label': 'Periode 4 (Oct-Des)', 'value': 'period4'}],                    
                value='period1',
                clearable=False,
                style={'width' : '50%', 'margin': '0 auto'}
            )
    ], style={'padding' : '10px', 'text-align' : 'center'}),

    # Main Content
    html.Div([
        # Map
        html.Div([
            dcc.Graph(
                id='temperature-map',
                figure=fig_map,
                style={'height' : '600px'}
            )
        ], className="six columns", style={
            'box-shadow' : '0 4px 8px 0 rgba(0,0,0,0.1)',
            'border-radius' : '10px',
            'background' : 'white',
            'padding' : '10px'
        }),

        # Plot
        html.Div([
            dcc.RadioItems(
                id='chart-type',
                options=[
                    {'label': 'Line Graph', 'value': 'line'},
                    {'label': 'Bar Chart', 'value': 'bar'}
                ],
                value='line',
                inline=True,
                labelStyle={
                    'margin':'0 15px',
                    'font-weight':'bold'
                }
            ),
            dcc.Graph(
                id='temperature-plot',
                style={'height':'550px'}
            )
        ], className="six columns", style={
            'box-shadow': '0 4px 8px 0 rgba(0,0,0,0.1)',
            'border-radius':'10px',
            'background':'white',
            'padding':'15px'
        })
    ], className="row", style={
        'max-width':'1800px',
        'margin':'0 auto',
        'padding':'10px'
    })
],style={
    'font-family':'Arial, sans-serif',
    'max-width': '100%',
    'margin':'0 auto',
    'padding':'20px',
    'background':'liniear-gradient(to bottom, #f5f7fa 0%, #e4e8eb 100%))',
    'min-height':'100vh'
})

# =========CALLBACK===============
@app.callback(
    [Output('temperature-map', 'figure'),
     Output('temperature-plot', 'figure')],
    [Input('period-dropdown', 'value'),
     Input('chart-type', 'value')]
)
def update_all(selected_period, chart_type):
    try:
        
        period_column = f'Suhu_bangunan_{selected_period}mean'
        min_temp = gdf[period_column].min()
        max_temp = gdf[period_column].max()
        
        updated_map = px.scatter_mapbox(
            gdf,
            lat="centroid_lat",
            lon="centroid_lon",
            color=period_column,
            color_continuous_scale=custom_color_scale,
            range_color=(min_temp, max_temp),
            zoom=15
        ).update_layout(
            mapbox_style="carto-positron",
            margin={"r":0,"t":0,"l":0,"b":0}
        )

        
        periods = ['Periode 1', 'Periode 2', 'Periode 3', 'Periode 4']
        period_values = [f'period{i+1}' for i in range(4)]
        
        
        avg_temps = [
            gdf[f'Suhu_bangunan_period{i+1}mean'].mean() 
            for i in range(4)
        ]
        
        if chart_type == 'line':
            fig = px.line(
                x=periods,
                y=avg_temps,
                title='Average Temperature Trend per Period',
                labels={'x': 'Period', 'y': 'Temp (°C)'},
                markers=True
            ).update_traces(
                line=dict(color='#f03b20', width=3),
                marker=dict(size=10)
            )
        else:
            fig = px.bar(
                x=periods,
                y=avg_temps,
                title='Temperature Comparison per Period',
                labels={'x': 'Period', 'y': 'Temp (°C)'},
                color=periods,
                color_discrete_sequence=[
                    '#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20'  
                ]
            )
        
        
        fig.update_layout(
            plot_bgcolor='white',
            paper_bgcolor='#f9f9f9',
            hovermode='x unified'
        )
        
        return updated_map, fig

    except Exception as e:
        print(f"Error: {str(e)}")
        error_fig = go.Figure().update_layout(
            title=f"Error: {str(e)}",
            xaxis={'visible': False},
            yaxis={'visible': False}
        )
        return updated_map, error_fig  
            

if __name__ == '__main__':
    app.run(debug=True)